## Read in MTA turnstile data

In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as mat
import datetime as dt
sys.path.append('../src')
import mta_data_clean as mc

In [2]:
# Read the datafiles
data_path = '../data/'
pickle_file = 'mta_data.pickle'
'''
data_file_list = ['turnstile_191102.txt',
                 'turnstile_191109.txt',
                 'turnstile_191116.txt',
                 'turnstile_191130.txt',
                 'turnstile_191207.txt',
                 'turnstile_191214.txt',
                 'turnstile_191221.txt',
                 'turnstile_191228.txt'
                 'turnstile_200104.txt']
'''
data_file_list = ['turnstile_191102.txt',
                 'turnstile_191005.txt']
booth_key_filename = 'Remote-Booth-Station.csv'

In [3]:
df = mc.mta_data_pickle_read(pickle_file)
df.reset_index(inplace=True, drop=True)

In [4]:
df

,station,division,date,time,desc,entries,exits,weekday,deltaEntry,deltaExit
0,59 ST,BMT,2019-03-30,00:00:00,REGULAR,6999064,2373568,Saturday,0,0
1,59 ST,BMT,2019-03-30,04:00:00,REGULAR,6999084,2373576,Saturday,20,8
2,59 ST,BMT,2019-03-30,08:00:00,REGULAR,6999107,2373622,Saturday,23,46
3,59 ST,BMT,2019-03-30,12:00:00,REGULAR,6999214,2373710,Saturday,107,88
4,59 ST,BMT,2019-03-30,16:00:00,REGULAR,6999451,2373781,Saturday,237,71
...,...,...,...,...,...,...,...,...,...,...
1225019,RIT-ROOSEVELT,RIT,2019-05-10,05:00:00,REGULAR,5554,376,Friday,0,0
1225020,RIT-ROOSEVELT,RIT,2019-05-10,09:00:00,REGULAR,5554,376,Friday,0,0
1225021,RIT-ROOSEVELT,RIT,2019-05-10,13:00:00,REGULAR,5554,376,Friday,0,0
1225022,RIT-ROOSEVELT,RIT,2019-05-10,17:00:00,REGULAR,5554,376,Friday,0,0


## Read in Manhattan subway station location data

In [5]:
# Import data containing locatios of Manhattan subway stations only
new_stations = pd.read_csv('http://web.mta.info/developers/data/nyct/subway/Stations.csv')

In [6]:
new_stations.head()

,Station ID,Complex ID,GTFS Stop ID,Division,Line,Stop Name,Borough,Daytime Routes,Structure,GTFS Latitude,GTFS Longitude,North Direction Label,South Direction Label
0,1,1,R01,BMT,Astoria,Astoria - Ditmars Blvd,Q,N W,Elevated,40.775036,-73.912034,NaN,Manhattan
1,2,2,R03,BMT,Astoria,Astoria Blvd,Q,N W,Elevated,40.770258,-73.917843,Ditmars Blvd,Manhattan
2,3,3,R04,BMT,Astoria,30 Av,Q,N W,Elevated,40.766779,-73.921479,Astoria - Ditmars Blvd,Manhattan
3,4,4,R05,BMT,Astoria,Broadway,Q,N W,Elevated,40.761820,-73.925508,Astoria - Ditmars Blvd,Manhattan
4,5,5,R06,BMT,Astoria,36 Av,Q,N W,Elevated,40.756804,-73.929575,Astoria - Ditmars Blvd,Manhattan


In [7]:
new_stations.Borough.value_counts()

Bk    169
M     153
Q      83
Bx     70
SI     21
Name: Borough, dtype: int64

In [8]:
# Filter DataFrame to include only Manhattan stations
manhattan_stations = new_stations[new_stations['Borough'] == 'M']
manhattan_stations.shape

(153, 13)

In [9]:
manhattan_stations.columns

Index(['Station ID', 'Complex ID', 'GTFS Stop ID', 'Division', 'Line',
       'Stop Name', 'Borough', 'Daytime Routes', 'Structure', 'GTFS Latitude',
       'GTFS Longitude', 'North Direction Label', 'South Direction Label'],
      dtype='object')

In [10]:
manhattan_stations['Stop Name'].value_counts()

Canal St                               6
23 St                                  5
125 St                                 4
96 St                                  4
Times Sq - 42 St                       4
                                      ..
77 St                                  1
Prince St                              1
1 Av                                   1
Bowling Green                          1
42 St - Port Authority Bus Terminal    1
Name: Stop Name, Length: 99, dtype: int64

## Visualize Manhattan subway station locations

In [11]:
# Import folium package to generate map to plot subway locations
import folium

In [12]:
def generateBaseMap(default_location=[40.693943, -73.985880], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [13]:
# Visualize density of subway locations
from folium.plugins import HeatMap
manhattan_copy = manhattan_stations.copy()
manhattan_copy['count'] = 1
basemap = generateBaseMap()
HeatMap(data=manhattan_copy[['GTFS Latitude', 'GTFS Longitude', 'count']].groupby(['GTFS Latitude', 'GTFS Longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(basemap);

In [14]:
basemap

In [15]:
# Visualize physical subway locations
for lat,lon,station in zip(manhattan_copy['GTFS Latitude'],manhattan_copy['GTFS Longitude'],manhattan_copy['Stop Name']):
     folium.CircleMarker([lat, lon],
                            popup=station,
                            color='b',
                            fill=True,
                            fill_opacity=0.7,
                           ).add_to(basemap)

In [16]:
basemap

## Merge stations and turnstile data

In [17]:
# Create a copy of the turnstiles data
df_copy = df.copy()

In [18]:
# Change the station names to title case in order to merge with 
# Manhattan station locations
df_copy['station'] = df_copy['station'].str.title()
df_copy.head()

,station,division,date,time,desc,entries,exits,weekday,deltaEntry,deltaExit
0,59 St,BMT,2019-03-30,00:00:00,REGULAR,6999064,2373568,Saturday,0,0
1,59 St,BMT,2019-03-30,04:00:00,REGULAR,6999084,2373576,Saturday,20,8
2,59 St,BMT,2019-03-30,08:00:00,REGULAR,6999107,2373622,Saturday,23,46
3,59 St,BMT,2019-03-30,12:00:00,REGULAR,6999214,2373710,Saturday,107,88
4,59 St,BMT,2019-03-30,16:00:00,REGULAR,6999451,2373781,Saturday,237,71


In [19]:
# Rename columns to lowercaser and underscores, for easier analysis
manhattan_copy.rename(columns=lambda x: x.strip().lower().replace(' ', '_'), inplace=True)

In [20]:
manhattan_copy.head()

,station_id,complex_id,gtfs_stop_id,division,line,stop_name,borough,daytime_routes,structure,gtfs_latitude,gtfs_longitude,north_direction_label,south_direction_label,count
6,7,613,R11,BMT,Astoria,Lexington Av/59 St,M,N W R,Subway,40.762660,-73.967258,Queens,Downtown & Brooklyn,1
7,8,8,R13,BMT,Astoria,5 Av/59 St,M,N W R,Subway,40.764811,-73.973347,Queens,Downtown & Brooklyn,1
8,9,9,R14,BMT,Broadway - Brighton,57 St - 7 Av,M,N Q R W,Subway,40.764664,-73.980658,Uptown & Queens,Downtown & Brooklyn,1
9,10,10,R15,BMT,Broadway - Brighton,49 St,M,N R W,Subway,40.759901,-73.984139,Uptown & Queens,Downtown & Brooklyn,1
10,11,611,R16,BMT,Broadway - Brighton,Times Sq - 42 St,M,N Q R W,Subway,40.754672,-73.986754,Uptown & Queens,Downtown & Brooklyn,1


In [21]:
# Isolate only columns to be used in the analysis
manhattan_copy = manhattan_copy.loc[:, ['stop_name', 'gtfs_latitude', 'gtfs_longitude']]

In [22]:
# Merge Manhattan station geodata with turnstile data
combined_data = pd.merge(df_copy, manhattan_copy, left_on='station', right_on='stop_name')

In [23]:
combined_data.head()

,station,division,date,time,desc,entries,exits,weekday,deltaEntry,deltaExit,stop_name,gtfs_latitude,gtfs_longitude
0,59 St,BMT,2019-03-30,00:00:00,REGULAR,6999064,2373568,Saturday,0,0,59 St,40.762526,-73.967967
1,59 St,BMT,2019-03-30,04:00:00,REGULAR,6999084,2373576,Saturday,20,8,59 St,40.762526,-73.967967
2,59 St,BMT,2019-03-30,08:00:00,REGULAR,6999107,2373622,Saturday,23,46,59 St,40.762526,-73.967967
3,59 St,BMT,2019-03-30,12:00:00,REGULAR,6999214,2373710,Saturday,107,88,59 St,40.762526,-73.967967
4,59 St,BMT,2019-03-30,16:00:00,REGULAR,6999451,2373781,Saturday,237,71,59 St,40.762526,-73.967967


## Read in location data of tech and Women's Health companies

In [24]:
tech_file = 'tech_ny.csv'
womens_health_file = 'womenshealth_ny.csv'

In [25]:
tech_companies = pd.read_csv(data_path + tech_file)
tech_companies.head()

,name,address,latitude,longitude
0,Facebook,"770 Broadway, New York, NY 10003",40.730701,-73.991531
1,Amazon,"450 W 33rd St, New York, NY 10001",40.753052,-73.999443
2,Google,"111 8th Ave, New York, NY 10011",40.798939,-73.024742
3,Icreon,"597 5th Ave 12th floor, New York, NY 10017",40.822520,-73.807920
4,IBM,"590 Madison Ave, New York, NY 10022",40.762090,-73.972790


In [26]:
womens_health_companies = pd.read_csv(data_path + womens_health_file)
womens_health_companies.head()

,name,address,latitude,longitude
0,Manhattan Women's Health,"993 Park Ave, New York, NY 10028",40.77852,-73.95791
1,Tribeca Women's Health,"21 Murray Street 3rd Floor, New York, NY 10007",40.71329,-74.00786
2,Manhattan Women's Health & Wellness,"51 E 25th St Ste 451, New York, NY 10010",40.74202,-73.98616
3,Manhattan Women's Health & Wellness,"55 W 17th St STE 104, New York, NY 10011",40.73921,-73.99475
4,Every Woman Wellness,"1375 Broadway #506, New York, NY 10018",40.75257,-73.98789


## Visualize each layer of detail on basemaps - traffic density (total), subway station locations, company locations

In [27]:
import folium

In [28]:
# Generate blank base map
def generateBaseMap(default_location=[40.758896, -73.985130], default_zoom_start=12):
    base_map = folium.Map(location=default_location, control_scale=True, zoom_start=default_zoom_start)
    return base_map

In [29]:
combined_data.head()

,station,division,date,time,desc,entries,exits,weekday,deltaEntry,deltaExit,stop_name,gtfs_latitude,gtfs_longitude
0,59 St,BMT,2019-03-30,00:00:00,REGULAR,6999064,2373568,Saturday,0,0,59 St,40.762526,-73.967967
1,59 St,BMT,2019-03-30,04:00:00,REGULAR,6999084,2373576,Saturday,20,8,59 St,40.762526,-73.967967
2,59 St,BMT,2019-03-30,08:00:00,REGULAR,6999107,2373622,Saturday,23,46,59 St,40.762526,-73.967967
3,59 St,BMT,2019-03-30,12:00:00,REGULAR,6999214,2373710,Saturday,107,88,59 St,40.762526,-73.967967
4,59 St,BMT,2019-03-30,16:00:00,REGULAR,6999451,2373781,Saturday,237,71,59 St,40.762526,-73.967967


### Subway Stations Map

In [30]:
stations_map = generateBaseMap()
for lat,lon,station in zip(manhattan_copy['gtfs_latitude'],manhattan_copy['gtfs_longitude'],manhattan_copy['stop_name']):
     folium.CircleMarker([lat, lon],
                            popup=station,
                            color='b',
                            radius=2.5,
                            fill=True,
                            fill_opacity=0.8,
                           ).add_to(stations_map)

In [31]:
stations_map

### Company Locations Map

Tech Companies

In [32]:
tech_map = generateBaseMap(default_zoom_start=12.5)
for lat,lon,company in zip(tech_companies['latitude'],tech_companies['longitude'],tech_companies['name']):
     folium.Marker([lat, lon],
                        popup=company,
                        icon=folium.Icon(color='darkblue'),
                        radius=2.5,
                        fill=True,
                        fill_opacity=0.8,
                       ).add_to(tech_map)

In [33]:
tech_map

Women's Health Map

In [34]:
womens_health_map = generateBaseMap()
for lat,lon,company in zip(womens_health_companies['latitude'],womens_health_companies['longitude'],womens_health_companies['name']):
     folium.Marker([lat, lon],
                        popup=company,
                        icon=folium.Icon(color='red'),
                        radius=2.5,
                        fill=True,
                        fill_opacity=0.8,
                       ).add_to(womens_health_map)

In [35]:
womens_health_map

### Combined tech and womens health maps

In [36]:
for lat,lon,company in zip(womens_health_companies['latitude'],womens_health_companies['longitude'],womens_health_companies['name']):
     folium.Marker([lat, lon],
                        popup=company,
                        icon=folium.Icon(color='red'),
                        radius=2.5,
                        fill=True,
                        fill_opacity=0.8,
                       ).add_to(tech_map)

In [37]:
tech_map

### Entries Map

In [38]:
from folium.plugins import HeatMap
combined_copy = combined_data.copy()
entries_map = generateBaseMap()
HeatMap(data=combined_copy[['gtfs_latitude', 'gtfs_longitude', 'deltaEntry']].groupby(['gtfs_latitude', 'gtfs_longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(entries_map);

In [39]:
entries_map

In [40]:
for lat,lon,station in zip(manhattan_copy['gtfs_latitude'],manhattan_copy['gtfs_longitude'],manhattan_copy['stop_name']):
     folium.CircleMarker([lat, lon],
                            popup=station,
                            color='b',
                            radius=2.5,
                            fill=True,
                            fill_opacity=0.8,
                           ).add_to(entries_map)

In [41]:
entries_map

### Exits Map

In [42]:
from folium.plugins import HeatMap
combined_copy = combined_data.copy()
exits_map = generateBaseMap()
HeatMap(data=combined_copy[['gtfs_latitude', 'gtfs_longitude', 'deltaExit']].groupby(['gtfs_latitude', 'gtfs_longitude']).sum().reset_index().values.tolist(), radius=8, max_zoom=13).add_to(exits_map);

In [43]:
exits_map

In [44]:
for lat,lon,station in zip(manhattan_copy['gtfs_latitude'],manhattan_copy['gtfs_longitude'],manhattan_copy['stop_name']):
     folium.CircleMarker([lat, lon],
                            popup=station,
                            color='b',
                            radius=2.5,
                            fill=True,
                            fill_opacity=0.8,
                           ).add_to(exits_map)

In [45]:
exits_map

### Combined Traffic Map + Layers

In [46]:
from folium.plugins import HeatMap
combined_copy = combined_data.copy()
traffic_map = generateBaseMap()
combined_copy['total_traffic'] = combined_copy['deltaExit'] + combined_copy['deltaEntry']
HeatMap(data=combined_copy[['gtfs_latitude', 'gtfs_longitude', 'total_traffic']].groupby(['gtfs_latitude', 'gtfs_longitude']).sum().reset_index().values.tolist(), radius=10, max_zoom=13).add_to(traffic_map);

In [47]:
traffic_map

### Add stations

In [48]:
for lat,lon,station in zip(manhattan_copy['gtfs_latitude'],manhattan_copy['gtfs_longitude'],manhattan_copy['stop_name']):
     folium.CircleMarker([lat, lon],
                            popup=station,
                            color='b',
                            radius=2.5,
                            fill=True,
                            fill_opacity=0.8,
                           ).add_to(traffic_map)

In [49]:
traffic_map

### Add companies

In [50]:
tech_map = generateBaseMap(default_zoom_start=12.5)
for lat,lon,company in zip(tech_companies['latitude'],tech_companies['longitude'],tech_companies['name']):
     folium.Marker([lat, lon],
                        popup=company,
                        color = 'blue',
                        radius=8,
                        fill=True,
                        fill_opacity=0.8,
                       ).add_to(tech_map)

In [51]:
for lat,lon,company in zip(womens_health_companies['latitude'],womens_health_companies['longitude'],womens_health_companies['name']):
     folium.Marker([lat, lon],
                        popup=company,
                        color = 'red',
                        radius=8,
                        fill=True,
                        icon=folium.Icon(color='red'),
                        fill_opacity=0.8,
                       ).add_to(tech_map)

In [52]:
for lat,lon,station in zip(manhattan_copy['gtfs_latitude'],manhattan_copy['gtfs_longitude'],manhattan_copy['stop_name']):
     folium.CircleMarker([lat, lon],
                            popup=station,
                            color='b',
                            radius=5,
                            fill=True,
                            fill_opacity=1,
                           ).add_to(tech_map)

In [53]:
from folium.plugins import HeatMap
combined_copy = combined_data.copy()
combined_copy['total_traffic'] = combined_copy['deltaExit'] + combined_copy['deltaEntry']
HeatMap(data=combined_copy[['gtfs_latitude', 'gtfs_longitude', 'total_traffic']].groupby(['gtfs_latitude', 'gtfs_longitude']).sum().reset_index().values.tolist(), radius=12, max_zoom=13).add_to(tech_map);

In [54]:
tech_map

## Visualize selected stations based on traffic volume analysis

### Read in selected stations

In [55]:
busy_stations = mc.mta_data_pickle_read('mta_busy_stations.pickle')
busy_stations.reset_index(inplace=True, drop=True)

In [56]:
busy_stations.head()

,station,division,date,time,desc,entries,exits,weekday,deltaEntry,deltaExit
0,59 ST,BMT,2019-03-30,00:00:00,REGULAR,6999064,2373568,Saturday,0,0
1,59 ST,BMT,2019-03-30,04:00:00,REGULAR,6999084,2373576,Saturday,20,8
2,59 ST,BMT,2019-03-30,08:00:00,REGULAR,6999107,2373622,Saturday,23,46
3,59 ST,BMT,2019-03-30,12:00:00,REGULAR,6999214,2373710,Saturday,107,88
4,59 ST,BMT,2019-03-30,16:00:00,REGULAR,6999451,2373781,Saturday,237,71


### Merge selected stations data with Manhattan station locations

In [57]:
busy_copy = busy_stations.copy()
busy_copy['station'] = busy_copy['station'].str.title()
busy_copy.head()

,station,division,date,time,desc,entries,exits,weekday,deltaEntry,deltaExit
0,59 St,BMT,2019-03-30,00:00:00,REGULAR,6999064,2373568,Saturday,0,0
1,59 St,BMT,2019-03-30,04:00:00,REGULAR,6999084,2373576,Saturday,20,8
2,59 St,BMT,2019-03-30,08:00:00,REGULAR,6999107,2373622,Saturday,23,46
3,59 St,BMT,2019-03-30,12:00:00,REGULAR,6999214,2373710,Saturday,107,88
4,59 St,BMT,2019-03-30,16:00:00,REGULAR,6999451,2373781,Saturday,237,71


In [58]:
# Load in all station data
station_file = 'NYC_Transit_Subway_Entrance_And_Exit_Data.csv'
station_locations = pd.read_csv(data_path+station_file)
station_locations.head()

,Division,Line,Station Name,Station Latitude,Station Longitude,Route1,Route2,Route3,Route4,Route5,...,ADA,ADA Notes,Free Crossover,North South Street,East West Street,Corner,Entrance Latitude,Entrance Longitude,Station Location,Entrance Location
0,BMT,4 Avenue,25th St,40.660397,-73.998091,R,NaN,NaN,NaN,NaN,...,False,NaN,False,4th Ave,25th St,SW,40.660489,-73.998220,"(40.660397, -73.998091)","(40.660489, -73.99822)"
1,BMT,4 Avenue,25th St,40.660397,-73.998091,R,NaN,NaN,NaN,NaN,...,False,NaN,False,4th Ave,25th St,SE,40.660323,-73.997952,"(40.660397, -73.998091)","(40.660323, -73.997952)"
2,BMT,4 Avenue,36th St,40.655144,-74.003549,N,R,NaN,NaN,NaN,...,False,NaN,True,4th Ave,36th St,NW,40.654676,-74.004306,"(40.655144, -74.003549)","(40.654676, -74.004306)"
3,BMT,4 Avenue,36th St,40.655144,-74.003549,N,R,NaN,NaN,NaN,...,False,NaN,True,4th Ave,36th St,NE,40.654365,-74.004113,"(40.655144, -74.003549)","(40.654365, -74.004113)"
4,BMT,4 Avenue,36th St,40.655144,-74.003549,N,R,NaN,NaN,NaN,...,False,NaN,True,4th Ave,36th St,NW,40.654490,-74.004499,"(40.655144, -74.003549)","(40.65449, -74.004499)"


In [59]:
stations_copy = station_locations.copy()

In [60]:
busy_stations_merge = pd.merge(busy_copy, stations_copy, left_on='station', right_on='Station Name')
busy_stations_merge.head()

,station,division,date,time,desc,entries,exits,weekday,deltaEntry,deltaExit,...,ADA,ADA Notes,Free Crossover,North South Street,East West Street,Corner,Entrance Latitude,Entrance Longitude,Station Location,Entrance Location
0,59 St,BMT,2019-03-30,00:00:00,REGULAR,6999064,2373568,Saturday,0,0,...,False,NaN,True,4th Ave,59th St,NW,40.641606,-74.017897,"(40.641362, -74.017881)","(40.641606, -74.017897)"
1,59 St,BMT,2019-03-30,00:00:00,REGULAR,6999064,2373568,Saturday,0,0,...,False,NaN,True,4th Ave,59th St,NE,40.641435,-74.017561,"(40.641362, -74.017881)","(40.641435, -74.017561)"
2,59 St,BMT,2019-03-30,00:00:00,REGULAR,6999064,2373568,Saturday,0,0,...,False,NaN,True,4th Ave,59th St,NW,40.641778,-74.017722,"(40.641362, -74.017881)","(40.641778, -74.017722)"
3,59 St,BMT,2019-03-30,00:00:00,REGULAR,6999064,2373568,Saturday,0,0,...,False,NaN,True,4th Ave,59th St,NE,40.641633,-74.017362,"(40.641362, -74.017881)","(40.641633, -74.017362)"
4,59 St,BMT,2019-03-30,00:00:00,REGULAR,6999064,2373568,Saturday,0,0,...,False,NaN,True,4th Ave,60th St,SW,40.640682,-74.018857,"(40.641362, -74.017881)","(40.640682, -74.018857)"


### Visualize locations on map layer

In [61]:
busy_locations = busy_stations_merge.groupby('station', as_index=False)['Entrance Latitude', 'Entrance Longitude'].mean()
busy_locations.head()

,station,Entrance Latitude,Entrance Longitude
0,125 St,40.804387,-73.937384
1,14 St,40.737925,-73.996479
2,14 St-Union Sq,40.734488,-73.989825
3,23 St,40.746123,-73.997754
4,28 St,40.747048,-73.993482


In [62]:
busy_map = generateBaseMap()
for lat,lon,station in zip(busy_locations['Entrance Latitude'],busy_locations['Entrance Longitude'],busy_locations['station']):
     folium.CircleMarker([lat, lon],
                            popup=station,
                            color='red',
                            radius=10,
                            fill=True,
                            fill_opacity=0.8,
                           ).add_to(busy_map)

In [63]:
busy_map

In [64]:
for lat,lon,company in zip(tech_companies['latitude'],tech_companies['longitude'],tech_companies['name']):
     folium.Marker([lat, lon],
                        popup=company,
                        radius=2.5,
                        fill=True,
                        fill_opacity=0.8,
                       ).add_to(busy_map)

In [65]:
for lat,lon,company in zip(womens_health_companies['latitude'],womens_health_companies['longitude'],womens_health_companies['name']):
     folium.Marker([lat, lon],
                        popup=company,
                        icon=folium.Icon(color='red'),
                        radius=2.5,
                        fill=True,
                        fill_opacity=0.8,
                       ).add_to(busy_map)

In [66]:
busy_map

### Select stations based on location and traffic and plot

In [67]:
# Selected stations based on location and traffic volume
chosen_stations = ['42 St-Port Auth', 'Path New Wtc', '14 St', '28 St', '42 St-Bryant Pk', '47-50 Sts Rock', '86 St', 'Chambers St', '34 St-Penn St', 'Grd Cntrl-42 St', '34 St-Herald Sq', '23 St', '14 St-Union Sq', 'Fulton St']

In [68]:
# Filter data set by chosen station and replot
chosen = busy_locations[busy_locations['station'].isin(chosen_stations)]

In [69]:
chosen_map = generateBaseMap()
for lat,lon,station in zip(chosen['Entrance Latitude'],chosen['Entrance Longitude'],chosen['station']):
     folium.CircleMarker([lat, lon],
                            popup=station,
                            color='red',
                            radius=10,
                            fill=True,
                            fill_opacity=0.8,
                           ).add_to(chosen_map)

In [70]:
for lat,lon,company in zip(tech_companies['latitude'],tech_companies['longitude'],tech_companies['name']):
     folium.Marker([lat, lon],
                        popup=company,
                        radius=2.5,
                        fill=True,
                        fill_opacity=0.2,
                       ).add_to(chosen_map)

In [71]:
for lat,lon,company in zip(womens_health_companies['latitude'],womens_health_companies['longitude'],womens_health_companies['name']):
     folium.Marker([lat, lon],
                        popup=company,
                        icon=folium.Icon(color='red'),
                        radius=2.5,
                        fill=True,
                        fill_opacity=0.2,
                       ).add_to(chosen_map)

In [72]:
chosen_map

In [86]:
chosen_only_map = generateBaseMap()
for lat,lon,station in zip(busy_locations['Entrance Latitude'],busy_locations['Entrance Longitude'],busy_locations['station']):
     folium.CircleMarker([lat, lon],
                            popup=station,
                            color='b',
                            radius=5,
                            fill=True,
                            fill_opacity=0.6,
                           ).add_to(chosen_only_map)

In [87]:
for lat,lon,station in zip(chosen['Entrance Latitude'],chosen['Entrance Longitude'],chosen['station']):
     folium.CircleMarker([lat, lon],
                            popup=station,
                            color='red',
                            radius=10,
                            fill=True,
                            fill_opacity=0.8,
                           ).add_to(chosen_only_map)

In [88]:
chosen_only_map